In [2]:
import models
import datasets
import torch.nn as nn
import torch
from tqdm import tqdm
import torchvision
from torch.utils.data import DataLoader
test_dataset = datasets.helen_data.Helen()

In [2]:
test_data = DataLoader(test_dataset,batch_size=2,shuffle=False)


In [3]:
net = models.RCNet.RCNet().cuda()
net.load_state_dict(torch.load('/home/zelin/csrnet/Baseline/RCNet/checkpoints/rcnet_050000.pt'))

<All keys matched successfully>

In [14]:
from glob import glob
from PIL import Image
import torchvision
net.eval()
for path in glob('/home/zelin/csrnet/Dataset/CelebA/*'):
    hr = Image.open(path)
    lr = hr.resize((24,24),Image.BILINEAR).resize((192,192),Image.BILINEAR)
    lr = torchvision.transforms.ToTensor()(lr).float().cuda()
    with torch.no_grad():
        coarse_sr,refine_sr,lds = net(lr.unsqueeze(0))
        torchvision.utils.save_image(refine_sr[-1],'results/celeba_/%s'%path.split('/')[-1].replace('jpg','png'))

In [4]:
from glob import glob
from PIL import Image
import torchvision
net.eval()
for path in glob('/home/zelin/csrnet/Dataset/helen/*'):
    hr = Image.open(path)
    lr = hr.resize((24,24),Image.BILINEAR).resize((192,192),Image.BILINEAR)
    lr = torchvision.transforms.ToTensor()(lr).float().cuda()
    with torch.no_grad():
        coarse_sr,refine_sr,lds = net(lr.unsqueeze(0))
        torchvision.utils.save_image(coarse_sr[-1],'results/helen_/%s'%path.split('/')[-1])

In [4]:
psnr = 0
for j,test_ in enumerate(test_data):
    net.eval()
    lr,hr = test_
    lr = lr.float().cuda()
    hr = hr.float().cuda()
    with torch.no_grad():
        coarse_sr,refine_sr,lds = net(lr)
        mse = ((refine_sr[-1]-hr)**2).mean(dim=[1,2,3])
        psnr += (10*torch.log10(1/mse)).mean()
    torchvision.utils.save_image(refine_sr[-1],

In [6]:
import cv2
import numpy as np
import math
def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()

def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(img1.shape[2]):
                ssims.append(ssim(img1[..., i], img2[..., i]))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

def bgr2ycbcr(img, only_y=True):
    '''same as matlab rgb2ycbcr
    only_y: only return Y channel
    Input:
        uint8, [0, 255]
        float, [0, 1]
    '''
    in_img_type = img.dtype
    img.astype(np.float32)
    if in_img_type != np.uint8:
        img *= 255.
    # convert
    if only_y:
        rlt = np.dot(img, [24.966, 128.553, 65.481]) / 255.0 + 16.0
    else:
        rlt = np.matmul(img, [[24.966, 112.0, -18.214], [128.553, -74.203, -93.786],
                              [65.481, -37.797, 112.0]]) / 255.0 + [16, 128, 128]
    if in_img_type == np.uint8:
        rlt = rlt.round()
    else:
        rlt /= 255.
    return rlt.astype(in_img_type)


In [7]:
import numpy as np
psnr_,ssim_,count = 0, 0, 0
for path in glob('/home/zelin/csrnet/Dataset/helen/*'):
    hr = np.array(Image.open(path))
    sr = np.array(Image.open(path.replace('Dataset/helen/','Baseline/RCNet/results/helen_/')))
    hr = bgr2ycbcr(hr)
    sr = bgr2ycbcr(sr)
    ssim_ += calculate_ssim(hr,sr)
    psnr_ += calculate_psnr(hr,sr)
    count += 1

psnr_/count,ssim_/count
    

(26.057283305605388, 0.7424278535132259)

In [23]:
import numpy as np
psnr_,ssim_,count = 0, 0, 0
for path in glob('/home/zelin/csrnet/Dataset/CelebA/*'):
    hr = np.array(Image.open(path))
    sr = np.array(Image.open(path.replace('Dataset/CelebA/','Baseline/RCNet/results/celeba/').replace('jpg','png')))
    hr = bgr2ycbcr(hr)
    sr = bgr2ycbcr(sr)
    ssim_ += calculate_ssim(hr,sr)
    psnr_ += calculate_psnr(hr,sr)
    count += 1

psnr_/count,ssim_/count
    

(25.176137249489248, 0.7077050334405388)